In [68]:
import requests
import json
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='white')
%matplotlib inline
from sklearn import decomposition
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [15]:
stat_url = "https://api.wotblitz.ru/wotb/encyclopedia/vehicles/?application_id=77636487a299e95a72583de66dba7a63"
response = json.loads(requests.get(stat_url).text)["data"]

In [18]:
b = response['62737']

In [132]:
def convert_to_array(data):
    cost_credit = None
    cost_gold = None
    if (data["cost"]):
        cost_credit = data["cost"]["price_credit"]
        cost_gold = data["cost"]["price_gold"]
        if cost_credit == 0:
            cost_credit = cost_gold * 400
    
    return [#data["tank_id"],
            data["name"]
            ,data["type"]
            ,data["nation"]
            ,data["tier"]
            ,1 if data["is_premium"] else 0
            ,cost_credit
#             ,cost_gold
            ,data["default_profile"]["weight"]
            ,data["default_profile"]["firepower"]
            ,data["default_profile"]["shot_efficiency"]
            ,data["default_profile"]["protection"]
            ,data["default_profile"]["maneuverability"]
            ,data["default_profile"]["speed_forward"]
            ,data["default_profile"]["speed_backward"]
            ,data["default_profile"]["max_ammo"]
            ,data["default_profile"]["hp"]
    
           ]

In [133]:
all_raw_data = [convert_to_array(x) for x in response.values()]

In [134]:
data = pd.DataFrame(all_raw_data, index=response.keys() , columns = ["name", "type", "nation", "tier", "is_prem", "price_credit", 
                                             "weight", "firepower", "shot_efficiency", "protection",
                                            "maneuverability", "speed_forward", "speed_backward", "max_ammo",
                                            "hp"])

In [138]:
nation_columns = pd.get_dummies(data["nation"])

In [160]:
pd.Categorical(data["type"]).categories

Index([u'AT-SPG', u'heavyTank', u'lightTank', u'mediumTank'], dtype='object')

In [146]:
type_column = pd.Categorical(data["type"]).codes

In [172]:
data["type"] = type_column
data = pd.concat([data.drop(["nation"], axis=1), nation_columns], axis=1)

ValueError: labels ['nation'] not contained in axis

In [173]:
data = data.drop(["name"], axis=1)

In [169]:
data.shape

(328, 22)

In [174]:
data.head()

,type,tier,is_prem,price_credit,weight,firepower,shot_efficiency,protection,maneuverability,speed_forward,...,max_ammo,hp,china,france,germany,japan,other,uk,usa,ussr
62737,2,8,1,2848000.0,23500,59,62,26,75,72,...,46,1250,0,0,1,0,0,0,0,0
8257,0,3,0,42500.0,2340,20,32,1,43,36,...,110,300,0,1,0,0,0,0,0,0
769,2,3,0,37000.0,14110,14,18,4,82,60,...,84,360,0,0,0,0,0,0,0,1
13329,1,4,0,170000.0,29930,18,8,23,25,35,...,90,540,0,0,1,0,0,0,0,0
2849,1,8,1,3040000.0,65138,45,83,74,27,35,...,34,1600,0,0,0,0,0,0,1,0


# PCA

In [175]:
pca = decomposition.PCA(n_components=2).fit(data)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').